In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
%matplotlib inline


In [2]:
train = pd.read_csv(r"data\train\train.csv")
test = pd.read_csv(r"data\test\test.csv")

In [3]:
train.head()

,id,text,author
0,id00001,Idris was well content with this resolve of mine.,MWS
1,id00002,"I was faint, even fainter than the hateful mod...",HPL
2,id00003,"Above all, I burn to know the incidents of you...",EAP
3,id00004,"He might see, perhaps, one or two points with ...",EAP
4,id00005,All obeyed the Lord Protector of dying England...,MWS


In [4]:
train.apply(lambda x: sum(x.isnull()),axis=0)

id        0
text      0
author    0
dtype: int64

In [5]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ME\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ME\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [7]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
lemm = WordNetLemmatizer()
ps=PorterStemmer()
def preprocessing(data):
    txt = data.lower() #1
    words = tokenizer.tokenize(txt) #2
    words = [w for w in words if not w in stop_words] #3
    words = [ps.stem(w) for w in words]
    words = ' '.join(words)#4
    return words

In [8]:
train_df=train
len(train)

19579

In [9]:
for i in range(0,len(train)):
    train_df["text"][i]= preprocessing(train["text"][i])


In [10]:
train_df['author'] = train_df.author.map({'EAP':0, 'HPL':1, 'MWS':2})

In [11]:
train_df.head()

,id,text,author
0,id00001,idri well content resolv mine,2
1,id00002,faint even fainter hate modern accurs citi made,1
2,id00003,burn know incid passag dark valley shadow,0
3,id00004,might see perhap one two point unusu clear nec...,0
4,id00005,obey lord protector die england look,2


In [12]:
X = train_df['text']
y = train_df['author']

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [14]:
# TO check if the authors are evenly distributed
print(y_train.value_counts())
print(y_test.value_counts())

0    6352
2    4815
1    4496
Name: author, dtype: int64
0    1548
2    1229
1    1139
Name: author, dtype: int64


# CountVectorizer :

Countvectorizer gives out the sparse matrix which contains the frequency of each word in respective documents/Data.

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')

In [16]:
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

# TF-IDF

Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics. To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies. Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus. This downscaling is called tf–idf for “Term Frequency times Inverse Document Frequency”.

https://manjunathhiremathm.wixsite.com/portfolio/blog-1/countvectorizer-v-s-tfidfvector

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [18]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Using Logistic Regression with CountVectorizer to evaluate the author


In [19]:
from sklearn.linear_model import LogisticRegression
LR_cv = LogisticRegression()
LR_cv.fit(X_train_cv, y_train)
y_test_LR = LR_cv.predict(X_test_cv)



In [20]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_test_LR))

0.789581205312


Confusion matrix : A confusion matrix is a matrix where we can see where the predicted values are, and where they should be.

In [21]:
from sklearn.metrics import confusion_matrix
cn = confusion_matrix(y_test, y_test_LR)
cn

array([[1315,  103,  130],
       [ 212,  849,   78],
       [ 223,   78,  928]])

Using Logistic Regression model with the entire dataset from "train.csv"

In [22]:
X_cv = cv.fit_transform(X)

In [23]:
LR_cv.fit(X_cv,y)
y_LR = LR_cv.predict(X_cv)
metrics.accuracy_score(y, y_LR)


0.93692221257469743

Predicting the author of the text given in the test.csv. Predicting the probabilities

In [24]:
test_cv = cv.transform(test['text'])


In [25]:
y_test_LR = LR_cv.predict(test_cv)

In [26]:
y_test_LR

array([2, 0, 0, ..., 0, 2, 0], dtype=int64)

In [27]:
y_test_LR_prob = LR_cv.predict_proba(test_cv)
y_test_LR_prob

array([[ 0.111541  ,  0.05499296,  0.83346603],
       [ 0.74820182,  0.21415462,  0.03764356],
       [ 0.75266949,  0.16106805,  0.08626246],
       ..., 
       [ 0.56147608,  0.10286553,  0.33565839],
       [ 0.47166303,  0.0189892 ,  0.50934777],
       [ 0.50337763,  0.49153645,  0.00508592]])

In [28]:
result_LR_cv = pd.DataFrame(y_test_LR_prob, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result_LR_cv.insert(0, 'id', test['id'])
result_LR_cv.insert(1, 'text', test['text'])
result_LR_cv.insert(2, 'Predicted',y_test_LR)
result_LR_cv.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.111541,0.054993,0.833466
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.748202,0.214155,0.037644
2,id00134,And when they had broken down the frail door t...,0,0.752669,0.161068,0.086262
3,id27757,While I was thinking how I should possibly man...,0,0.573506,0.343449,0.083046
4,id04081,I am not sure to what limit his knowledge may ...,0,0.851077,0.117539,0.031384


In [29]:
result_LR_cv['Predicted_Author'] = result_LR_cv.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result_LR_cv.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.111541,0.054993,0.833466,MWS
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.748202,0.214155,0.037644,EAP
2,id00134,And when they had broken down the frail door t...,0,0.752669,0.161068,0.086262,EAP
3,id27757,While I was thinking how I should possibly man...,0,0.573506,0.343449,0.083046,EAP
4,id04081,I am not sure to what limit his knowledge may ...,0,0.851077,0.117539,0.031384,EAP


In [30]:
from pandas import ExcelWriter

writer = ExcelWriter('Predicted Author.xlsx' , engine='xlsxwriter')
result_LR_cv.to_excel(writer,sheet_name='LogisticRegression_CountVec',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])


# Naive Bayes with CountVectorizer 

In [31]:
from sklearn.naive_bayes import MultinomialNB
NB_cv = MultinomialNB()
NB_cv.fit(X_train_cv, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
y_nb_cv = NB_cv.predict(X_test_cv)
print(metrics.accuracy_score(y_test, y_nb_cv))

0.82277834525


In [33]:
X_cv = cv.fit_transform(X)
X_cv

<19579x15322 sparse matrix of type '<class 'numpy.int64'>'
	with 222396 stored elements in Compressed Sparse Row format>

In [34]:
NB_cv.fit(X_cv,y)
y_nb_cv = NB_cv.predict(X_cv)
metrics.accuracy_score(y, y_nb_cv)

0.88227182184994124

In [35]:
test_cv = cv.transform(test['text'])
test_cv

<8392x15322 sparse matrix of type '<class 'numpy.int64'>'
	with 43644 stored elements in Compressed Sparse Row format>

In [36]:
y_pred_nb_cv = NB_cv.predict(test_cv)
y_pred_nb_cv

array([2, 0, 0, ..., 0, 2, 0], dtype=int64)

In [37]:
y_pred_prob_nb_cv = NB_cv.predict_proba(test_cv)
y_pred_prob_nb_cv

array([[  5.78020295e-03,   1.10145784e-02,   9.83205219e-01],
       [  9.87252695e-01,   1.23984564e-02,   3.48848452e-04],
       [  5.15246729e-01,   4.72347945e-01,   1.24053261e-02],
       ..., 
       [  6.98617621e-01,   7.80223543e-02,   2.23360024e-01],
       [  5.89312256e-02,   1.97645131e-03,   9.39092323e-01],
       [  5.79469385e-01,   4.17610417e-01,   2.92019781e-03]])

In [38]:
result = pd.DataFrame(y_pred_prob_nb_cv, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result.insert(0, 'id', test['id'])
result.insert(1, 'text', test['text'])
result.insert(2, 'Predicted',y_pred_nb_cv)
result.head()


,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.005780,0.011015,0.983205
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.987253,0.012398,0.000349
2,id00134,And when they had broken down the frail door t...,0,0.515247,0.472348,0.012405
3,id27757,While I was thinking how I should possibly man...,1,0.205567,0.782408,0.012026
4,id04081,I am not sure to what limit his knowledge may ...,0,0.941036,0.051086,0.007877


In [39]:
result['Predicted_Author'] = result.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.005780,0.011015,0.983205,MWS
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.987253,0.012398,0.000349,EAP
2,id00134,And when they had broken down the frail door t...,0,0.515247,0.472348,0.012405,EAP
3,id27757,While I was thinking how I should possibly man...,1,0.205567,0.782408,0.012026,HPL
4,id04081,I am not sure to what limit his knowledge may ...,0,0.941036,0.051086,0.007877,EAP


In [40]:
result.to_excel(writer,sheet_name='NavieBayes_CountVec',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

# SVM with countVectorizer

In [41]:
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.svm import SVC

In [42]:
svd = decomposition.TruncatedSVD(n_components=120)

In [43]:
svd.fit(X_train_cv)

TruncatedSVD(algorithm='randomized', n_components=120, n_iter=5,
       random_state=None, tol=0.0)

In [44]:
xtrain_svd = svd.transform(X_train_cv)
xtest_svd = svd.transform(X_test_cv)

In [45]:
# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xtest_svd)

In [46]:
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
y_pred_svd = clf.predict(xvalid_svd_scl)

In [48]:
print(metrics.accuracy_score(y_test, y_pred_svd))

0.662410623085


In [49]:
X_cv = cv.fit_transform(X)
X_cv
svd.fit(X_cv)
x_svd = svd.transform(X_cv)

In [50]:
scl.fit(x_svd)
x_svd = scl.transform(x_svd)

In [51]:
clf.fit(x_svd, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [52]:
y_svd = clf.predict(x_svd)

In [53]:
print(metrics.accuracy_score(y, y_svd))

0.783339292099


In [54]:
test_cv = cv.transform(test['text'])
test_cv
test_svd = svd.transform(test_cv)
test_svd = scl.transform(test_svd)

In [55]:
y_test_svd = clf.predict(test_svd)

In [56]:
y_test_prob_svd = clf.predict_proba(test_svd)


In [57]:
result_svd = pd.DataFrame(y_test_prob_svd, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result_svd.insert(0, 'id', test['id'])
result_svd.insert(1, 'text', test['text'])
result_svd.insert(2, 'Predicted',y_test_svd)
result_svd.head()


,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.010569,0.018409,0.971022
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.734341,0.177843,0.087817
2,id00134,And when they had broken down the frail door t...,1,0.336325,0.571543,0.092132
3,id27757,While I was thinking how I should possibly man...,0,0.752999,0.207357,0.039644
4,id04081,I am not sure to what limit his knowledge may ...,0,0.853674,0.110707,0.035619


In [58]:
result_svd['Predicted_Author'] = result_svd.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result_svd.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.010569,0.018409,0.971022,MWS
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.734341,0.177843,0.087817,EAP
2,id00134,And when they had broken down the frail door t...,1,0.336325,0.571543,0.092132,HPL
3,id27757,While I was thinking how I should possibly man...,0,0.752999,0.207357,0.039644,EAP
4,id04081,I am not sure to what limit his knowledge may ...,0,0.853674,0.110707,0.035619,EAP


In [59]:
result_svd.to_excel(writer,sheet_name='SVC_CountVec',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

# K-nearest neighbors with Count Vectorizer



In [60]:
from sklearn.neighbors import KNeighborsClassifier


In [61]:
KNN = KNeighborsClassifier(n_neighbors = 5,metric="minkowski",p=2)

In [62]:
KNN.fit(X_train_cv,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [63]:
y_pred_knn = KNN.predict(X_test_cv)

In [64]:
metrics.accuracy_score(y_test,y_pred_knn)

0.42441266598569971

In [65]:
X_cv = cv.fit_transform(X)
X_cv

<19579x15322 sparse matrix of type '<class 'numpy.int64'>'
	with 222396 stored elements in Compressed Sparse Row format>

In [66]:
KNN.fit(X_cv,y)
y_knn = KNN.predict(X_cv)
metrics.accuracy_score(y, y_knn)

0.61800909137341031

In [67]:
test_cv = cv.transform(test['text'])
test_cv

<8392x15322 sparse matrix of type '<class 'numpy.int64'>'
	with 43644 stored elements in Compressed Sparse Row format>

In [68]:
y_pred_knn = KNN.predict(test_cv)
y_pred_knn

array([0, 0, 0, ..., 1, 2, 0], dtype=int64)

In [69]:
y_pred_prob_knn = KNN.predict_proba(test_cv)
y_pred_prob_knn

array([[ 0.4,  0.2,  0.4],
       [ 0.8,  0.2,  0. ],
       [ 0.8,  0. ,  0.2],
       ..., 
       [ 0.2,  0.4,  0.4],
       [ 0.4,  0. ,  0.6],
       [ 0.8,  0. ,  0.2]])

In [70]:
result_knn = pd.DataFrame(y_pred_prob_knn, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result_knn.insert(0, 'id', test['id'])
result_knn.insert(1, 'text', test['text'])
result_knn.insert(2, 'Predicted',y_pred_knn)
result_knn.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",0,0.4,0.2,0.4
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.8,0.2,0.0
2,id00134,And when they had broken down the frail door t...,0,0.8,0.0,0.2
3,id27757,While I was thinking how I should possibly man...,2,0.4,0.0,0.6
4,id04081,I am not sure to what limit his knowledge may ...,0,0.4,0.2,0.4


In [71]:
result_knn['Predicted_Author'] = result_knn.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result_knn.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",0,0.4,0.2,0.4,EAP
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.8,0.2,0.0,EAP
2,id00134,And when they had broken down the frail door t...,0,0.8,0.0,0.2,EAP
3,id27757,While I was thinking how I should possibly man...,2,0.4,0.0,0.6,MWS
4,id04081,I am not sure to what limit his knowledge may ...,0,0.4,0.2,0.4,EAP


In [72]:
result_knn.to_excel(writer,sheet_name='KNN_CountVec',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

# Decision Tree with Count Vectorizer

In [73]:
from sklearn.tree import DecisionTreeClassifier

In [74]:
decisiontree = DecisionTreeClassifier(criterion='entropy',random_state=0)

In [75]:
decisiontree.fit(X_train_cv,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [76]:
y_pred_decision = decisiontree.predict(X_test_cv)

In [77]:
metrics.accuracy_score(y_test,y_pred_decision)

0.59346271705822262

In [78]:
X_cv = cv.fit_transform(X)
X_cv

<19579x15322 sparse matrix of type '<class 'numpy.int64'>'
	with 222396 stored elements in Compressed Sparse Row format>

In [79]:
decisiontree.fit(X_cv,y)
y_decisiontree = decisiontree.predict(X_cv)
metrics.accuracy_score(y, y_decisiontree)

0.99897849736963074

In [80]:
test_cv = cv.transform(test['text'])
test_cv

<8392x15322 sparse matrix of type '<class 'numpy.int64'>'
	with 43644 stored elements in Compressed Sparse Row format>

In [81]:
y_pred_decision = decisiontree.predict(test_cv)
y_pred_decision

array([2, 0, 1, ..., 0, 2, 0], dtype=int64)

In [82]:
y_pred_prob_decision = decisiontree.predict_proba(test_cv)
y_pred_prob_decision

array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       ..., 
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]])

In [83]:
result_decision = pd.DataFrame(y_pred_prob_decision, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result_decision.insert(0, 'id', test['id'])
result_decision.insert(1, 'text', test['text'])
result_decision.insert(2, 'Predicted',y_pred_decision)
result_decision.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.0,0.0,1.0
1,id24541,"If a fire wanted fanning, it could readily be ...",0,1.0,0.0,0.0
2,id00134,And when they had broken down the frail door t...,1,0.0,1.0,0.0
3,id27757,While I was thinking how I should possibly man...,2,0.0,0.0,1.0
4,id04081,I am not sure to what limit his knowledge may ...,0,1.0,0.0,0.0


In [84]:
result_decision['Predicted_Author'] = result_decision.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result_decision.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.0,0.0,1.0,MWS
1,id24541,"If a fire wanted fanning, it could readily be ...",0,1.0,0.0,0.0,EAP
2,id00134,And when they had broken down the frail door t...,1,0.0,1.0,0.0,HPL
3,id27757,While I was thinking how I should possibly man...,2,0.0,0.0,1.0,MWS
4,id04081,I am not sure to what limit his knowledge may ...,0,1.0,0.0,0.0,EAP


In [85]:
result_decision.to_excel(writer,sheet_name='DecisionTree_CountVec',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

# Random Forest with Count Vectorizer

In [86]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
RFC = RandomForestClassifier(n_estimators=15,criterion='entropy',random_state=0)

In [88]:
RFC.fit(X_train_cv,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=15, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [89]:
y_test_RFC=RFC.predict(X_test_cv)

In [90]:
metrics.accuracy_score(y_test,y_test_RFC)

0.6657303370786517

In [91]:
X_cv = cv.fit_transform(X)
X_cv

<19579x15322 sparse matrix of type '<class 'numpy.int64'>'
	with 222396 stored elements in Compressed Sparse Row format>

In [92]:
RFC.fit(X_cv,y)
y_RFC = RFC.predict(X_cv)
metrics.accuracy_score(y, y_RFC)

0.9948414117166352

In [93]:
test_cv = cv.transform(test['text'])
test_cv

<8392x15322 sparse matrix of type '<class 'numpy.int64'>'
	with 43644 stored elements in Compressed Sparse Row format>

In [94]:
y_pred_RFC = RFC.predict(test_cv)
y_pred_RFC

array([2, 0, 1, ..., 0, 2, 0], dtype=int64)

In [95]:
y_pred_prob_RFC = RFC.predict_proba(test_cv)
y_pred_prob_RFC

array([[ 0.        ,  0.13333333,  0.86666667],
       [ 0.93333333,  0.        ,  0.06666667],
       [ 0.33333333,  0.6       ,  0.06666667],
       ..., 
       [ 0.78863826,  0.04498747,  0.16637427],
       [ 0.13333333,  0.        ,  0.86666667],
       [ 0.86666667,  0.13333333,  0.        ]])

In [96]:
result_RFC = pd.DataFrame(y_pred_prob_RFC, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result_RFC.insert(0, 'id', test['id'])
result_RFC.insert(1, 'text', test['text'])
result_RFC.insert(2, 'Predicted',y_pred_RFC)
result_RFC.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.000000,0.133333,0.866667
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.933333,0.000000,0.066667
2,id00134,And when they had broken down the frail door t...,1,0.333333,0.600000,0.066667
3,id27757,While I was thinking how I should possibly man...,1,0.200000,0.533333,0.266667
4,id04081,I am not sure to what limit his knowledge may ...,0,0.855556,0.101389,0.043056


In [97]:
result_RFC['Predicted_Author'] = result_RFC.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result_RFC.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.000000,0.133333,0.866667,MWS
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.933333,0.000000,0.066667,EAP
2,id00134,And when they had broken down the frail door t...,1,0.333333,0.600000,0.066667,HPL
3,id27757,While I was thinking how I should possibly man...,1,0.200000,0.533333,0.266667,HPL
4,id04081,I am not sure to what limit his knowledge may ...,0,0.855556,0.101389,0.043056,EAP


In [98]:
result_RFC.to_excel(writer,sheet_name='RandomForest_CountVec',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

# TF-IDF

# Logistic Regression with Tf-idf

In [99]:
from sklearn.linear_model import LogisticRegression
LR_tfidf = LogisticRegression()
LR_tfidf.fit(X_train_tfidf, y_train)
y_test_LR = LR_tfidf.predict(X_test_tfidf)



In [100]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_test_LR))

0.801583248212


In [101]:
from sklearn.metrics import confusion_matrix
cn = confusion_matrix(y_test, y_test_LR)
cn

array([[1334,  101,  113],
       [ 203,  875,   61],
       [ 221,   78,  930]])

In [102]:
X_tfidf = tfidf.fit_transform(X)

In [103]:
LR_tfidf.fit(X_tfidf,y)
y_tfidf = LR_tfidf.predict(X_tfidf)
metrics.accuracy_score(y, y_tfidf)


0.89110781960263552

In [104]:
test_tfidf = tfidf.transform(test['text'])


In [105]:
y_test_LR = LR_tfidf.predict(test_tfidf)

In [106]:
y_test_LR

array([2, 0, 0, ..., 0, 2, 0], dtype=int64)

In [107]:
y_test_LR_prob = LR_tfidf.predict_proba(test_tfidf)
y_test_LR_prob

array([[ 0.20973185,  0.08483839,  0.70542976],
       [ 0.47603747,  0.30873769,  0.21522483],
       [ 0.5305958 ,  0.32520859,  0.14419562],
       ..., 
       [ 0.41648784,  0.22405744,  0.35945471],
       [ 0.24707586,  0.112402  ,  0.64052214],
       [ 0.54314605,  0.34780422,  0.10904974]])

In [108]:
result_LR_tfidf = pd.DataFrame(y_test_LR_prob, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result_LR_tfidf.insert(0, 'id', test['id'])
result_LR_tfidf.insert(1, 'text', test['text'])
result_LR_tfidf.insert(2, 'Predicted',y_test_LR)
result_LR_tfidf.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.209732,0.084838,0.705430
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.476037,0.308738,0.215225
2,id00134,And when they had broken down the frail door t...,0,0.530596,0.325209,0.144196
3,id27757,While I was thinking how I should possibly man...,1,0.401243,0.417736,0.181021
4,id04081,I am not sure to what limit his knowledge may ...,0,0.790123,0.122302,0.087575


In [109]:
result_LR_tfidf['Predicted_Author'] = result_LR_tfidf.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result_LR_tfidf.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.209732,0.084838,0.705430,MWS
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.476037,0.308738,0.215225,EAP
2,id00134,And when they had broken down the frail door t...,0,0.530596,0.325209,0.144196,EAP
3,id27757,While I was thinking how I should possibly man...,1,0.401243,0.417736,0.181021,HPL
4,id04081,I am not sure to what limit his knowledge may ...,0,0.790123,0.122302,0.087575,EAP


In [110]:
#from pandas import ExcelWriter

#writer = ExcelWriter('Predicted Author.xlsx' , engine='xlsxwriter')
result_LR_tfidf.to_excel(writer,sheet_name='LogisticRegression_TF-IDF',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

# Navie Bayes With TF-IDF

In [111]:
from sklearn.naive_bayes import MultinomialNB
NB_tfidf = MultinomialNB()
NB_tfidf.fit(X_train_tfidf, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [112]:
y_nb_tfidf = NB_tfidf.predict(X_test_tfidf)
print(metrics.accuracy_score(y_test, y_nb_tfidf))

0.813329928498


In [113]:
X_tfidf = tfidf.fit_transform(X)
X_tfidf

<19579x15455 sparse matrix of type '<class 'numpy.float64'>'
	with 248708 stored elements in Compressed Sparse Row format>

In [114]:
NB_tfidf.fit(X_tfidf,y)
y_nb_tfidf = NB_tfidf.predict(X_tfidf)
metrics.accuracy_score(y, y_nb_tfidf)

0.88554063026712293

In [115]:
test_tfidf = tfidf.transform(test['text'])
test_tfidf

<8392x15455 sparse matrix of type '<class 'numpy.float64'>'
	with 65695 stored elements in Compressed Sparse Row format>

In [116]:
y_pred_nb_tfidf = NB_tfidf.predict(test_tfidf)
y_pred_nb_tfidf

array([2, 0, 0, ..., 0, 2, 0], dtype=int64)

In [117]:
y_pred_prob_nb_tfidf = NB_tfidf.predict_proba(test_tfidf)
y_pred_prob_nb_tfidf

array([[ 0.17020005,  0.12610037,  0.70369957],
       [ 0.41183583,  0.32855372,  0.25961044],
       [ 0.41578313,  0.36300928,  0.22120759],
       ..., 
       [ 0.52699082,  0.17569507,  0.2973141 ],
       [ 0.2253899 ,  0.14293414,  0.63167596],
       [ 0.44629432,  0.33572895,  0.21797673]])

In [118]:
result = pd.DataFrame(y_pred_prob_nb_tfidf, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result.insert(0, 'id', test['id'])
result.insert(1, 'text', test['text'])
result.insert(2, 'Predicted',y_pred_nb_tfidf)
result.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.170200,0.126100,0.703700
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.411836,0.328554,0.259610
2,id00134,And when they had broken down the frail door t...,0,0.415783,0.363009,0.221208
3,id27757,While I was thinking how I should possibly man...,1,0.266230,0.546470,0.187299
4,id04081,I am not sure to what limit his knowledge may ...,0,0.746673,0.142763,0.110564


In [119]:
result['Predicted_Author'] = result.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.170200,0.126100,0.703700,MWS
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.411836,0.328554,0.259610,EAP
2,id00134,And when they had broken down the frail door t...,0,0.415783,0.363009,0.221208,EAP
3,id27757,While I was thinking how I should possibly man...,1,0.266230,0.546470,0.187299,HPL
4,id04081,I am not sure to what limit his knowledge may ...,0,0.746673,0.142763,0.110564,EAP


In [120]:
result.to_excel(writer,sheet_name='NavieBayes_TF-IDF',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

# SVM with TFIDF


In [121]:
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.svm import SVC

In [122]:
svd = decomposition.TruncatedSVD(n_components=120)

In [123]:
svd.fit(X_train_tfidf)

TruncatedSVD(algorithm='randomized', n_components=120, n_iter=5,
       random_state=None, tol=0.0)

In [124]:
xtrain_svd = svd.transform(X_train_tfidf)
xtest_svd = svd.transform(X_test_tfidf)

In [125]:
# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xtest_svd)

In [126]:
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [127]:
y_pred_svd = clf.predict(xvalid_svd_scl)

In [128]:
print(metrics.accuracy_score(y_test, y_pred_svd))

0.707099080695


In [129]:
X_tfidf = tfidf.fit_transform(X)
X_tfidf
svd.fit(X_tfidf)
x_svd = svd.transform(X_tfidf)

In [130]:
scl.fit(x_svd)
x_svd = scl.transform(x_svd)

In [131]:
clf.fit(x_svd, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [132]:
y_svd = clf.predict(x_svd)

In [133]:
print(metrics.accuracy_score(y, y_svd))

0.832064967567


In [134]:
test_tfidf = tfidf.transform(test['text'])
test_tfidf
test_svd = svd.transform(test_tfidf)
test_svd = scl.transform(test_svd)

In [135]:
y_test_svd = clf.predict(test_svd)

In [136]:
y_test_prob_svd = clf.predict_proba(test_svd)

In [137]:
result_svd = pd.DataFrame(y_test_prob_svd, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result_svd.insert(0, 'id', test['id'])
result_svd.insert(1, 'text', test['text'])
result_svd.insert(2, 'Predicted',y_test_svd)
result_svd.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.043424,0.046499,0.910077
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.592022,0.356950,0.051027
2,id00134,And when they had broken down the frail door t...,1,0.132171,0.851194,0.016635
3,id27757,While I was thinking how I should possibly man...,0,0.660679,0.322661,0.016660
4,id04081,I am not sure to what limit his knowledge may ...,0,0.817397,0.062078,0.120525


In [138]:
result_svd['Predicted_Author'] = result_svd.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result_svd.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.043424,0.046499,0.910077,MWS
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.592022,0.356950,0.051027,EAP
2,id00134,And when they had broken down the frail door t...,1,0.132171,0.851194,0.016635,HPL
3,id27757,While I was thinking how I should possibly man...,0,0.660679,0.322661,0.016660,EAP
4,id04081,I am not sure to what limit his knowledge may ...,0,0.817397,0.062078,0.120525,EAP


In [139]:
result_svd.to_excel(writer,sheet_name='SVC_TFIDF',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

# K-nearest neighbors with TFIDF

In [140]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 5,metric="minkowski",p=2)

In [141]:
KNN.fit(X_train_tfidf,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [142]:
y_pred_knn = KNN.predict(X_test_tfidf)

In [143]:
metrics.accuracy_score(y_test,y_pred_knn)

0.59346271705822262

In [144]:
X_tfidf = tfidf.fit_transform(X)
X_tfidf

<19579x15455 sparse matrix of type '<class 'numpy.float64'>'
	with 248708 stored elements in Compressed Sparse Row format>

In [145]:
KNN.fit(X_tfidf,y)
y_knn = KNN.predict(X_tfidf)
metrics.accuracy_score(y, y_knn)

0.41411716635170337

In [146]:
test_tfidf = tfidf.transform(test['text'])
test_tfidf

<8392x15455 sparse matrix of type '<class 'numpy.float64'>'
	with 65695 stored elements in Compressed Sparse Row format>

In [147]:
y_pred_knn = KNN.predict(test_tfidf)
y_pred_knn

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [148]:
y_pred_prob_knn = KNN.predict_proba(test_tfidf)
y_pred_prob_knn

array([[ 0.6,  0. ,  0.4],
       [ 0.6,  0.2,  0.2],
       [ 0.6,  0. ,  0.4],
       ..., 
       [ 0.8,  0. ,  0.2],
       [ 0.6,  0. ,  0.4],
       [ 0.6,  0. ,  0.4]])

In [149]:
result_knn = pd.DataFrame(y_pred_prob_knn, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result_knn.insert(0, 'id', test['id'])
result_knn.insert(1, 'text', test['text'])
result_knn.insert(2, 'Predicted',y_pred_knn)
result_knn.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",0,0.6,0.0,0.4
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.6,0.2,0.2
2,id00134,And when they had broken down the frail door t...,0,0.6,0.0,0.4
3,id27757,While I was thinking how I should possibly man...,0,0.6,0.2,0.2
4,id04081,I am not sure to what limit his knowledge may ...,0,0.8,0.0,0.2


In [150]:
result_knn['Predicted_Author'] = result_knn.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result_knn.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",0,0.6,0.0,0.4,EAP
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.6,0.2,0.2,EAP
2,id00134,And when they had broken down the frail door t...,0,0.6,0.0,0.4,EAP
3,id27757,While I was thinking how I should possibly man...,0,0.6,0.2,0.2,EAP
4,id04081,I am not sure to what limit his knowledge may ...,0,0.8,0.0,0.2,EAP


In [151]:
result_knn.to_excel(writer,sheet_name='KNN_TF-IDF',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

# Decision Tree with TF-IDF

In [152]:
from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier(criterion='entropy',random_state=0)

In [153]:
decisiontree.fit(X_train_tfidf,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [154]:
y_pred_decision = decisiontree.predict(X_test_tfidf)

In [155]:
metrics.accuracy_score(y_test,y_pred_decision)

0.59346271705822262

In [156]:
X_tfidf = tfidf.fit_transform(X)
X_tfidf

<19579x15455 sparse matrix of type '<class 'numpy.float64'>'
	with 248708 stored elements in Compressed Sparse Row format>

In [157]:
decisiontree.fit(X_tfidf,y)
y_decisiontree = decisiontree.predict(X_tfidf)
metrics.accuracy_score(y, y_decisiontree)

0.99969354921088927

In [158]:
test_tfidf = tfidf.transform(test['text'])
test_tfidf

<8392x15455 sparse matrix of type '<class 'numpy.float64'>'
	with 65695 stored elements in Compressed Sparse Row format>

In [159]:
y_pred_decision = decisiontree.predict(test_tfidf)
y_pred_decision

array([2, 0, 1, ..., 2, 2, 0], dtype=int64)

In [160]:
y_pred_prob_decision = decisiontree.predict_proba(test_tfidf)
y_pred_prob_decision

array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]])

In [161]:
result_decision = pd.DataFrame(y_pred_prob_decision, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result_decision.insert(0, 'id', test['id'])
result_decision.insert(1, 'text', test['text'])
result_decision.insert(2, 'Predicted',y_pred_decision)
result_decision.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.0,0.0,1.0
1,id24541,"If a fire wanted fanning, it could readily be ...",0,1.0,0.0,0.0
2,id00134,And when they had broken down the frail door t...,1,0.0,1.0,0.0
3,id27757,While I was thinking how I should possibly man...,2,0.0,0.0,1.0
4,id04081,I am not sure to what limit his knowledge may ...,0,1.0,0.0,0.0


In [162]:
result_decision['Predicted_Author'] = result_decision.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result_decision.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.0,0.0,1.0,MWS
1,id24541,"If a fire wanted fanning, it could readily be ...",0,1.0,0.0,0.0,EAP
2,id00134,And when they had broken down the frail door t...,1,0.0,1.0,0.0,HPL
3,id27757,While I was thinking how I should possibly man...,2,0.0,0.0,1.0,MWS
4,id04081,I am not sure to what limit his knowledge may ...,0,1.0,0.0,0.0,EAP


In [163]:
result_decision.to_excel(writer,sheet_name='DecisionTree_TFIDF',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

# Random Forest with TF-IDF

In [164]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=15,criterion='entropy',random_state=0)

In [165]:
RFC.fit(X_train_tfidf,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=15, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [166]:
y_test_RFC=RFC.predict(X_test_tfidf)
metrics.accuracy_score(y_test,y_test_RFC)

0.68335035750766093

In [167]:
X_tfidf = tfidf.fit_transform(X)
X_tfidf

<19579x15455 sparse matrix of type '<class 'numpy.float64'>'
	with 248708 stored elements in Compressed Sparse Row format>

In [168]:
RFC.fit(X_tfidf,y)
y_RFC = RFC.predict(X_tfidf)
metrics.accuracy_score(y, y_RFC)

0.99535216303181984

In [169]:
test_tfidf = tfidf.transform(test['text'])
test_tfidf

<8392x15455 sparse matrix of type '<class 'numpy.float64'>'
	with 65695 stored elements in Compressed Sparse Row format>

In [170]:
y_pred_RFC = RFC.predict(test_tfidf)
y_pred_RFC

array([2, 0, 0, ..., 0, 2, 0], dtype=int64)

In [171]:
y_pred_prob_RFC = RFC.predict_proba(test_tfidf)
y_pred_prob_RFC

array([[ 0.        ,  0.13333333,  0.86666667],
       [ 0.62222222,  0.13333333,  0.24444444],
       [ 0.46666667,  0.4       ,  0.13333333],
       ..., 
       [ 0.53333333,  0.06666667,  0.4       ],
       [ 0.2       ,  0.13333333,  0.66666667],
       [ 0.73333333,  0.13333333,  0.13333333]])

In [172]:
result_RFC = pd.DataFrame(y_pred_prob_RFC, columns=['EAP_Prob','HPL_Prob','MWS_Prob'])
result_RFC.insert(0, 'id', test['id'])
result_RFC.insert(1, 'text', test['text'])
result_RFC.insert(2, 'Predicted',y_pred_RFC)
result_RFC.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.000000,0.133333,0.866667
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.622222,0.133333,0.244444
2,id00134,And when they had broken down the frail door t...,0,0.466667,0.400000,0.133333
3,id27757,While I was thinking how I should possibly man...,1,0.266667,0.533333,0.200000
4,id04081,I am not sure to what limit his knowledge may ...,0,0.866667,0.000000,0.133333


In [173]:
result_RFC['Predicted_Author'] = result_RFC.Predicted.map({0:'EAP', 1:'HPL', 2:'MWS'})
result_RFC.head()

,id,text,Predicted,EAP_Prob,HPL_Prob,MWS_Prob,Predicted_Author
0,id02310,"Still, as I urged our leaving Ireland with suc...",2,0.000000,0.133333,0.866667,MWS
1,id24541,"If a fire wanted fanning, it could readily be ...",0,0.622222,0.133333,0.244444,EAP
2,id00134,And when they had broken down the frail door t...,0,0.466667,0.400000,0.133333,EAP
3,id27757,While I was thinking how I should possibly man...,1,0.266667,0.533333,0.200000,HPL
4,id04081,I am not sure to what limit his knowledge may ...,0,0.866667,0.000000,0.133333,EAP


In [174]:
result_RFC.to_excel(writer,sheet_name='RandomForest_TF-IDF',index=False,  columns=['id','text', 'Predicted_Author','EAP_Prob','HPL_Prob',
'MWS_Prob'])

In [175]:
writer.save()